In [1]:
import requests
from requests.auth import HTTPBasicAuth
from classes import BaseLoader
from datalayer import MysqlDataManager
from settings import datapi_url, datapi_username, datapi_passwd, mariadb_config, alpha_key
import logging
import requests
from time import sleep


def get_price_rows(data: dict):
    """
    For getting the price rows from the response data
    """

    price_rows = []
    symbol = full_data['Meta Data']['2. Symbol']
    
    for day_item in full_data["Time Series (Daily)"].items():
        datum = day_item[0]
        opening = float(day_item[1]["1. open"])
        high = float(day_item[1]["2. high"])
        low = float(day_item[1]["3. low"])
        closing = float(day_item[1]["4. close"])
        price_row = tuple([symbol, datum, opening, high, low, closing])
        price_rows.append(price_row)

    return price_rows

## Get the daily data from datapi

In [2]:
# Initialize the logger
logger = logging.getLogger(__name__)
logging.basicConfig(filename="loading_datapi.log", encoding="utf-8", level=logging.ERROR,
                    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', datefmt='%Y-%m-%d, %H:%M:%S')

# Initialize the local database connection
loader = BaseLoader(mariadb_config)
res = requests.get(datapi_url, auth=HTTPBasicAuth(datapi_username, datapi_passwd))

# request for data
if 299 >= res.status_code >= 200:
    data_source = res.json()
    try:
        result = loader.upload(data_source)
    except Exception as err:
        logger.error("Something goes wrong in loading_datapi: %s", err)
        raise err
    else:
        print("Result from upload to api:", result)

        # request for delete the data on the remote server
        res = requests.delete(datapi_url, auth=HTTPBasicAuth(datapi_username, datapi_passwd))
        print("Status Code from DELETE request:", res.status_code)

elif not (299 >= res.status_code >= 200):
    print(f"Status Code: {res.status_code} occured!")
    print(res.json()["response"])


Result from upload to api: 45
Status Code from DELETE request: 200


## Get full data from alphavantage

In [4]:
select_query= """SELECT
    s.symbol
FROM
    stocks AS s
LEFT JOIN
    stock_price sp ON s.symbol = sp.symbol
WHERE
    sp.symbol IS NULL;"""

# Set the connection with database and alphavantage
url_template = "https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={}&outputsize=full&apikey={}"
sql_query = "insert into stock_price values (%s, %s, %s, %s, %s, %s);"
data_manager = MysqlDataManager(mariadb_config)
symbols = data_manager.select(select_query)

# Insert full data for each symbol in database

for item in symbols:
    full_data = requests.get(url_template.format(item[0], alpha_key)).json()
    
    price_rows = get_price_rows(full_data)

    sleep(1)
    rows_affected = data_manager.query(sql_query, price_rows)
    print(f"For {item[0]} rows affected: {rows_affected}")

For 3868Rows affected: 3868
For 5220Rows affected: 5220
For 5220Rows affected: 5220
For 5219Rows affected: 5219
For 4719Rows affected: 4719


### Get full data per symbol

In [2]:
# Initialize important objects
symbol = input("Symbol: ").upper()
url_template = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&outputsize=full&apikey={alpha_key}"
sql_query = "insert into stock_price values (%s, %s, %s, %s, %s, %s);"
data_manager = MysqlDataManager(mariadb_config)

# Start the request
full_data = requests.get(url_template.format(symbol, alpha_key)).json()
price_rows = get_price_rows(full_data)
result = data_manager.query(sql_query, price_rows)

print("Rows affected", result)

Symbol:  SDIV


IntegrityError: 1452 (23000): Cannot add or update a child row: a foreign key constraint fails (`finance`.`stock_price`, CONSTRAINT `stock_price_stocks_FK` FOREIGN KEY (`symbol`) REFERENCES `stocks` (`symbol`) ON DELETE CASCADE ON UPDATE CASCADE)